In [2]:
import pandas as pd

def least_unit_cost(requirements, setup_cost, holding_cost, initial_inventory):
    periods = len(requirements)

    i = 0
    ending_inventory = 0
    total_cost = 0
    detailed_plan = []

    # Adjust the first period's requirement based on initial inventory
    if periods > 0:
        effective_requirement = max(0, requirements[0] - initial_inventory)
        requirements = [effective_requirement] + requirements[1:]

    while i < periods:
        min_cost = float('inf')
        best_j = i
        cumulative_demand = 0

        for j in range(i, periods):
            cumulative_demand += requirements[j]
            holding_costs = sum(holding_cost * (k - i) * requirements[k] for k in range(i+1, j+1))
            luc_cost = (setup_cost + holding_costs) / cumulative_demand

            if luc_cost > min_cost:
                break

            min_cost = luc_cost
            best_j = j

        total_production = sum(requirements[i:best_j+1])

        if total_production > 0:
            total_cost += setup_cost  # Add setup cost once for the batch

            for k in range(i, best_j+1):
                if k == i:
                    production_amount = total_production
                else:
                    production_amount = 0

                inventory_cost = ending_inventory * holding_cost
                total_cost += inventory_cost
                ending_inventory += production_amount - requirements[k]
                ending_inventory = max(0, ending_inventory)

                detailed_plan.append((k + 1, requirements[k], production_amount, ending_inventory))

        i = best_j + 1

    df = pd.DataFrame(detailed_plan, columns=['Period', 'Requirements', 'Production', 'Ending Inventory'])
    return df, total_cost

# Example usage
requirements = [6, 12, 4, 8, 15, 25, 20, 5, 10, 20, 5, 20]
setup_cost = 40
holding_cost = 1
initial_inventory = 4

print("Least Unit Cost with Initial Inventory:")
df, cost = least_unit_cost(requirements, setup_cost, holding_cost, initial_inventory)
print(df.to_string(index=False))
print("Total Cost: ", cost)

Least Unit Cost with Initial Inventory:
 Period  Requirements  Production  Ending Inventory
      1             2          26                24
      2            12           0                12
      3             4           0                 8
      4             8           0                 0
      5            15          40                25
      6            25           0                 0
      7            20          25                 5
      8             5           0                 0
      9            10          35                25
     10            20           0                 5
     11             5           0                 0
     12            20          20                 0
Total Cost:  304
